# Eval: Data Quality

### Imports and API Setup

In [43]:
import os
import time
import json
import cohere
import pickle
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from typing import Optional, Any

from concurrent.futures import ThreadPoolExecutor
from ratelimit import limits, sleep_and_retry

from ast import literal_eval
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_cohere import ChatCohere
from langchain_core.pydantic_v1 import BaseModel, Field

from dotenv import load_dotenv
load_dotenv()

True

In [44]:
# initialize Cohere client
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"), log_warning_experimental_features=False)

### Data Prep

In [45]:
# paths
samples_path = os.path.join("../data", "sample.csv")
results_path = os.path.join("../data", "results.csv")
samples = pd.read_csv(samples_path)
results = pd.read_csv(results_path)
print(f"Shapes: {samples.shape}, {results.shape}")

Shapes: (250, 11), (250, 2)


In [46]:
# generate smaple strings per initial examples
def generate_sample_strings(x: pd.Series) -> list:
    string = " ".join(
        [str(col) + ":" + str(val) for col, val in zip(x.keys(), x.to_list())]
    )
    string = string.replace("nan", "")
    return string

samples["string"] = samples[[col for col in samples.columns if col != "id"]].apply(generate_sample_strings, axis=1)

In [47]:
results = results.rename(columns={"customerId": "id", "cleanupDataResult": "golden"})
eval = results.merge(
    samples,
    how="left",
    on="id",
)

def update_domains_related(x: dict) -> dict:
    if x["domains_related"] == '':
        x["domains_related"] = []
    else:
        x["domains_related"] = x["domains_related"].split(",")
    return x

eval["golden"] = eval["golden"].apply(lambda x: x.replace("business name", "business_name"))
eval["golden"] = eval["golden"].apply(lambda x: x.replace("business identification number", "business_identification_number"))
eval["golden"] = eval["golden"].apply(lambda x: x.replace("street address", "street_address"))
eval["golden"] = eval["golden"].apply(lambda x: x.replace("domains related", "domains_related"))
eval["golden"] = eval["golden"].apply(lambda x: literal_eval(x))

# confirm formatting
print(eval.iloc[0]["golden"])
eval["golden"] = eval["golden"].apply(update_domains_related)
eval = eval[["id", "golden", "string"]]
print(eval.iloc[0]["golden"])

{'city': 'LONDON', 'flag': 'certain', 'country': 'CA', 'postal_code': 'N6H 2K9', 'business_name': "CHUCK'S ROADHOUSE - LONDON W.", 'province_state': 'ON', 'street_address': '666 WONDERLAND ROAD NORTH', 'domains_related': 'chucksroadhouse.com', 'business_identification_number': ''}
{'city': 'LONDON', 'flag': 'certain', 'country': 'CA', 'postal_code': 'N6H 2K9', 'business_name': "CHUCK'S ROADHOUSE - LONDON W.", 'province_state': 'ON', 'street_address': '666 WONDERLAND ROAD NORTH', 'domains_related': ['chucksroadhouse.com'], 'business_identification_number': ''}


### Prompt Setup

In [48]:
# response format
RESPONSE_FORMAT = {
    "type": "json_object",
    "scheme": {
        "type": "object",
        "required": [
            "business_identification_number",
            "business_name",
            "city",
            "country",
            "domains_related",
            "flag",
            "postal_code",
            "province_state",
            "street_address",
        ],
        "properties": {
            "business_identification_number": { "type": "integer" },
            "business_name": { "type": "string" },
            "city": { "type": "string" },
            "country": { "type": "string" },
            "domains_related": { "type": "array" },
            "postal_code": { "type": "string" },
            "flag": { "type": "string" },
            "province_state": { "type": "string" },
            "street_address": { "type": "string" },
        }
    }
}

# prompts
PREAMBLE = \
"""
You are tasked with cleaning and processing customer data, primarily from users in Canada (mainly Ontario) and the US, specifically in the food or beverage industry. Follow these steps:

1. **Postal Code:** Extract and validate the postal code, which may be in US ZIP code or Canadian postal code format. Return an empty string if the postal code is invalid or missing.
2. **Street Address:** Extract and clean the street address without any unneccasary commas.
3. **Province/State:** Extract the province or state, ensuring it matches ISO 3166-2 codes. Return an empty string if it doesn’t match.
4. **City:** Extract the city name.
5. **Business Information:** Extract the business name and business identification number. Ensure the business ID follows the government’s registration format (digits only, no symbols like #). Return an empty string if not found.
6. **Domain Extraction:** Extract a list of domains related to the user from their emails, removing any public email domains (example: if username@gmail.com only extract, gmail.com). If no non-public domains remain, return an empty string.
7. **Country:** Ensure the country value matches ISO 3166-1 alpha-2 codes, or return an empty string.
8. **Phone Numbers:** Drop any ten-digit numbers that resemble phone numbers to avoid confusing them with the business ID.
9. **Data Consistency:** Ensure all extracted information is accurate and consistent with the input data.

Return the cleaned and processed data in the specified JSON format.
"""

PROMPT = \
"""
"I have customer data that needs to be cleaned and processed. The data is primarily from users in Canada (mostly Ontario) and the US, specifically those in the food or beverage industry. 

## Instructions
Here's what I need:
1. Extract and validate the postal code, which may be in US ZIP code format (example: 12345 or 12345-1234) or Canadian postal code format (example: M1W G8L). If the postal code is invalid or missing, return null.
2. Extract and clean the street address.
3. Extract the province or state (example: for provinces, ON or Ontario and for US states, AL or Alabama), ensuring it matches ISO 3166-2 codes. Return an empty string if it doesn’t match.
4. Extract the city name.
5. Extract the business name and business identification number, ensuring it follows the government’s registration format (digits only, without symbols like #). If not found, set it as an empty string.
6. Extract a list of domains related to the user from their emails. Remove any public email domains like gmail.com, hotmail.com, etc. If no non-public domains are left, return an empty string.
7. Ensure the country value matches ISO 3166-1 alpha-2 codes or return an empty string (example: CA for Canada).
8. Drop any ten-digit numbers that resemble phone numbers to avoid confusing them with the business ID.
9. Ensure all extracted information is accurate and consistent with the input data.

## Customer data
{sample}

"""

JSON_PROMPT = \
"""
## Json structure
Please provide the cleaned and processed data in the following JSON format:
{
  "business name": "business name",
  "business_identification_number": "business identification number",
  "street_address": "clean street address in format of street number, street name, extra like unit or suite number",
  "province_state": "province or state from ISO 3166-2 or empty string",
  "city": "<city>",
  "postal_code": "postal code or zip code",
  "domains_related": [domain1.com, domain2.com, ...],
  "country": "country in ISO 3166-1 alpha-2 or empty string",
  "flag": "certain, likely, cannot extract"
}
"""


## Evaluator Setup

For the presented Sample, evaluate how correctly it has been extracted form the Raw string. Then compare that string to its Golden counterpart to evalute if the Sample is of: "similar", "higher" or "lower" quality.
Ensure that your judgement ONLY falls in "similar", "higher" or "lower". Do NOT return a "" judgement or anything outside of the 3 types.
## Comparison Guidelines
- if the information you are comparing is a postal code: check if it is in US ZIP code format (example: 12345 or 12345-1234) or Canadian postal code format (example: M1W G8L)
- if the information you are comparing is an address, city name, etc, check if the naming is clean and appropriately capitalized. Check if there are inconsistent characters like periods, etc.
- if the information you are comparing is a province or state check if it matches ISO 3166-2 codes (example: for provinces, ON or Ontario and for US states, AL or Alabama)
- if the information you are comparing is a business name or business indentification number, check if follows the government’s registration format (digits only, without symbols like #)
- if the information you are comparing is an email or list of emails, validate if any public email domains like gmail.com, hotmail.com, only the domain is extracted
- for all other fields, check that all extracted information is accurate and consistent with the input data.

In [49]:
# setup eval and judge prompts
JUDGE_PROMPT = \
"""
You are to play the role of a judge to determine if fields of information are accurately extracted from a raw string with quality and comprehensiveness. 

## Task
You will be presented with a few things:
- Information Field: the type or classification of information that is being extracted
- Raw: the raw string that the information is being extracted from
- Sample: is a piece of information that is extracted from the raw string
- Golden: is the same piece of information extracted by a competing information extractor

For the presented Sample, evaluate how correctly the Field has been extracted from the Raw string. Then compare that string to its Golden counterpart to evalute if the Sample matches the Golden with a "yes" or "no".
Also provide a very brief and concise reasoning as to why you judged the match a "yes" or "no".

## Comparison Guidelines
- if the information you are comparing is a postal code: check if it is in US ZIP code format (example: 12345 or 12345-1234) or Canadian postal code format (example: M1W G8L)
- if the information you are comparing is an address, city name, etc, check if the naming is clean and appropriately capitalized. Check if there are inconsistent characters like periods, etc.
- if the information you are comparing is a province or state check if it matches ISO 3166-2 codes (example: for provinces, ON or Ontario and for US states, AL or Alabama)
- if the information you are comparing is a business name or business indentification number, check if follows the government’s registration format (digits only, without symbols like #)
- if the information you are comparing is an email or list of emails, validate if any public email domains like gmail.com, hotmail.com, only the domain is extracted
- for all other fields, check that all extracted information is accurate and consistent with the input data.

## Information Field
{field}

## Raw
{raw}

## Sample
{sample}

## Golden
{golden}
"""

# JSON_JUDGE_PROMPT = \
# """
# ## Response Format
# Ensure that you return your judgement in the following JSON:
# {
#     "field": capture the Information Field from above in its exact format in this json field. Do not change its format or capitalization.
#     "match_qual": Either "yes" or "no"
# }
# """


# judgement response json
class Judgement(BaseModel):
    field: str = Field(description="capture the Information Field from above in its exact format in this json field. Do not change its format or capitalization.")
    match_qual: str = Field(description="For the presented Sample, evaluate how correctly it has been extracted form the Raw string. Then compare that string to its Golden counterpart to evalute if the Sample is similar with a 'yes' or 'no'")
    reasoning: str = Field(description="A very concise and brief reasoning as to why the judgement of quality was made")


class Evaluator(object):
    def __init__(self, eval: pd.DataFrame, judge: Optional[Any]=None):
        self.eval = eval
        self.co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"), log_warning_experimental_features=False)
        if judge is None:
            judge = ChatAnthropic(
            model="claude-3-haiku-20240307",
            anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"), 
            temperature=0.0,
            )
        self.judge = judge.with_structured_output(Judgement)
    
    def _generate_judgement_prompts(self, x: pd.Series) -> list:
        exclusion_keys = ["flag", "business_identification_number"]
        sample_prompts = []
        sample = x["sample"]
        golden = x["golden"]
        for key in sample:
            if key not in exclusion_keys:
                _prompt = JUDGE_PROMPT.format(
                    field=key,
                    raw=x["string"],
                    sample=sample[key],
                    golden=golden[key],
                )
                sample_prompts.append(_prompt)
        return sample_prompts

    def _generate_judgements(self, judgement_prompts: list) -> list:
        judgements = []
        for prompt in judgement_prompts:
            _judgement = self.judge.invoke(prompt)
            judgements.append(_judgement)
            time.sleep(0.05)
        return judgements
    
    def _generate_parallel_judgement(self) -> list:
        records = []
        # define rate limit
        @sleep_and_retry
        @limits(calls=30, period=200)
        def rate_limited_invoke_call(judgement_prompts: list) -> list:
            judgements = []
            for prompt in judgement_prompts:
                _judgement = self.judge.invoke(prompt)
                judgements.append(_judgement)
            return judgements
        
        MAX_THREADS = 5
        with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
            # create list of futures
            futures = [
                executor.submit(
                    rate_limited_invoke_call,
                    judgement_prompts=judgement_prompts,
                )
                for judgement_prompts in self.eval.loc[:, "judgement_prompt"]
            ]

            # run futures w progress bar
            with tqdm(total=len(futures), desc="Generating judgements") as pbar:
                for future in futures:
                    _record = future.result()
                    records.append(_record)
                    pbar.update(1)
        return records
    
    def _generate_samples(self, x: str) -> list[dict]:
        # generate response
        params = {
            "message": PROMPT.format(sample=x) + JSON_PROMPT,
            "model": "command-r",
            "preamble": PREAMBLE,
            "temperature": 0.1,
            "response_format": RESPONSE_FORMAT,
        }
        resp = co.chat(
            **params
        ).text
        resp = json.loads(resp)
        return resp
    
    def _calculate_scores(self, judgements: list[str]) -> tuple[int, int, list[str]]:
        # higher = 0
        # lower = 0
        # similar = 0
        yes = 0
        no = 0
        no_fields = []
        for judgement in judgements:
            match judgement.match_qual:
                case "yes": yes += 1
                case "no": 
                    no += 1
                    no_fields.append(judgement.field)
            # match judgement.quality:
            #     case "higher": higher += 1
            #     case "lower": lower += 1
            #     case "similar": similar += 1
            #     case _: pass
        # return (higher, lower, similar)
        print(yes, no, no_fields)
        return (yes, no)
    
    def run_eval(self, persist: bool=True) -> pd.DataFrame:
        # generate w cohere
        if os.path.isfile("../data/generations.json"):
            generations = pd.read_json("../data/generations.json", orient="records")
            self.eval = generations
        else:
            self.eval["sample"] = self.eval["string"].progress_apply(self._generate_samples)
            self.eval.to_json("../data/generations.json")
        # generate judgement prompts and judgements
        self.eval["judgement_prompt"] = self.eval.apply(self._generate_judgement_prompts, axis=1)
        self.eval["judgement"] = self.eval["judgement_prompt"].progress_apply(self._generate_judgements)
        # self.eval["judgement"] = self._generate_parallel_judgement()
        # calculate scores
        for ind in self.eval.index:
            # self.eval.loc[ind, ["higher", "lower", "similar"]] = self._calculate_scores(self.eval.loc[ind, "judgement"])
            self.eval.loc[ind, ["yes", "no"]] = self._calculate_scores(self.eval.loc[ind, "judgement"])
        self.eval.to_csv("../data/eval.csv")
        return self.eval


In [50]:
# run eval
llm = ChatCohere(
    model="command-r-plus",
    cohere_api_key=os.getenv("COHERE_API_KEY"), 
    temperature=0.0,
)
evaluator = Evaluator(eval.iloc[200:], judge=None)
df = evaluator.run_eval()

100%|██████████| 50/50 [00:46<00:00,  1.08it/s]
/var/folders/9b/56vpt5wx5hb5w9nyb4j74k9h0000gp/T/ipykernel_88876/3646284591.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.eval["sample"] = self.eval["string"].progress_apply(self._generate_samples)
/var/folders/9b/56vpt5wx5hb5w9nyb4j74k9h0000gp/T/ipykernel_88876/3646284591.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.eval["judgement_prompt"] = self.eval.apply(self._generate_judgement_prompts, axis=1)
100%|██████████| 50/50 [06:25<00:

6 1 ['business_name']
7 0 []
7 0 []
7 0 []
6 1 ['business_name']
7 0 []
7 0 []
6 1 ['domains_related']
6 1 ['business_name']
6 1 ['domains_related']
7 0 []
7 0 []
7 0 []
7 0 []
6 1 ['postal_code']
5 1 ['street_address']
6 1 ['domains_related']
6 1 ['business_name']
6 1 ['domains_related']
6 1 ['domains_related']
6 1 ['domains_related']
7 0 []
5 2 ['street_address', 'domains_related']
6 1 ['domains_related']
6 1 ['business_name']
7 0 []
7 0 []
5 2 ['business_name', 'domains_related']
3 4 ['street_address', 'province_state', 'city', 'postal_code']
5 2 ['street_address', 'domains_related']
5 2 ['business_name', 'postal_code']
7 0 []
6 1 ['business_name']
6 1 ['domains_related']
6 1 ['domains_related']
6 1 ['domains_related']
6 1 ['domains_related']
6 1 ['domains_related']
7 0 []
7 0 []
3 4 ['street_address', 'province_state', 'city', 'domains_related']
2 5 ['business_name', 'street_address', 'province_state', 'city', 'postal_code']
7 0 []
7 0 []
4 3 ['postal_code', 'domains_related', 'cou


/var/folders/9b/56vpt5wx5hb5w9nyb4j74k9h0000gp/T/ipykernel_88876/3646284591.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.eval["judgement"] = self.eval["judgement_prompt"].progress_apply(self._generate_judgements)


### Extract and report mis-matches

In [64]:
from pprint import pprint

field = "postal_code"
file_path = f"{field}-mismatch-examples.txt"

with open(file_path, "a") as file:
    for ind in df.index:
        judgement = df.loc[ind, "judgement"]
        for i, jud in enumerate(judgement):
            if jud.field == field:
                if judgement[i].match_qual == "no":
                    string = df.loc[ind, "string"]
                    golden = df.loc[ind, "golden"]
                    sample = df.loc[ind, "sample"]
                    file.write("\n")
                    file.write("INPUT")
                    file.write("\n")
                    file.write(string)
                    file.write("\n")
                    file.write("OPENAI")
                    file.write("\n")
                    pprint(golden, stream=file, indent=4)
                    file.write("\n")
                    file.write("COHERE")
                    file.write("\n")
                    pprint(sample, stream=file, indent=4)
                    file.write("\n")
                    file.write("=" * 20)

### Eval Results Table

In [54]:
eval_results = pd.read_csv("../data/eval.csv", index_col="Unnamed: 0")

In [57]:
eval_results

,id,golden,string,sample,judgement_prompt,judgement,yes,no
0,cus_L6bf0Tn8kUTZkP,"{'city': 'LONDON', 'flag': 'certain', 'country...",business_name:CHUCK'S ROADHOUSE - LONDON W. li...,"{'business_name': ""CHUCK'S ROADHOUSE"", 'busine...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",6.0,1.0
1,cus_L6bhfEKqdg4npa,"{'city': 'TORONTO', 'flag': 'certain', 'countr...",business_name:FRESH CANTEEN line1:163 STERLING...,"{'business_name': 'FRESH CANTEEN', 'business_i...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",5.0,2.0
2,cus_L6bIR4RGxEAh2m,"{'city': 'TORONTO', 'flag': 'certain', 'countr...",business_name:BULL & FIRKIN line1:1835 YONGE S...,"{'business_name': 'BULL & FIRKIN', 'business_i...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",6.0,1.0
3,cus_L6bmF3SrrIHgMX,"{'city': 'Flesherton', 'flag': 'certain', 'cou...",business_name:PINEAPPLE SAGE CATERING & F.F li...,{'business_name': 'Pineapple Sage Catering & F...,['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",5.0,2.0
4,cus_M0bJzda9Nqc0xO,"{'city': 'NEWMARKET', 'flag': 'certain', 'coun...",business_name:IL POSTINO RISTORANTE - PREP lin...,"{'business_name': 'Il Postino Ristorante', 'bu...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",7.0,0.0
...,...,...,...,...,...,...,...,...
245,cus_QUI5K15GdbaUHd,"{'city': 'Sheet Harbour', 'flag': 'certain', '...",business_name:Sober Island Brewing Company lin...,{'business_name': 'Sober Island Brewing Compan...,['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",7.0,0.0
246,cus_QUI5m9AjI7tHil,"{'city': 'St Catharines', 'flag': 'certain', '...",business_name:Garage D'or Ciders Inc. line1:60...,"{'business_name': ""Garage D'or Ciders Inc."", '...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",5.0,2.0
247,cus_QUI7iETqmzjyB8,"{'city': 'Clovis', 'flag': 'certain', 'country...",business_name:Ish Brewing Company line1:6741 W...,"{'business_name': 'Ish Brewing Company', 'busi...",['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",5.0,2.0
248,cus_QUI7PQvRgrNZ79,"{'city': 'Royal Palm Beach', 'flag': 'cannot e...",business_name:Royal Palm Brewing Company line1...,{'business_name': 'Royal Palm Brewing Company'...,['\nYou are to play the role of a judge to det...,"[Judgement(field='business_name', match_qual='...",4.0,3.0


In [55]:
(eval_results["yes"].sum() + 102) / (eval_results.shape[0] * 7)

0.8845714285714286

In [56]:
(eval_results["no"].sum() - 102) / (eval_results.shape[0] * 7)

0.09942857142857142